In [ ]:
import datetime as dt
from dataclasses import dataclass

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pynextsim.nextsim_mesh import NextsimMesh
from pysida.lib import get_triangulation, cleanup_triangulation, MeshFileList
from scipy.spatial import KDTree

In [ ]:
@dataclass
class Pair:
    x0: np.ndarray
    x1: np.ndarray
    y0: np.ndarray
    y1: np.ndarray
    d0: pd.Timestamp
    d1: pd.Timestamp

In [ ]:
@dataclass
class Coords:
    x0: np.ndarray
    x1: np.ndarray
    y0: np.ndarray
    y1: np.ndarray

In [ ]:
def get_coords_from_traj_new(df, rgps_date0, timedelta=3, min_time_diff=0.5, max_time_diff=5, **kwargs):
    dfd = df[(df.d >= rgps_date0) * (df.d <= rgps_date0 + dt.timedelta(timedelta))]
    im2_idx = np.unique(dfd.i)
    coords = {}
    for im2 in im2_idx:
        df2 = df[df.i == im2]
        d2 = df2.iloc[0].d

        gpi = (
            (df.d >= (d2 - dt.timedelta(max_time_diff))) *
            (df.d <= (d2 - dt.timedelta(min_time_diff)))
        )

        im1_idx = np.unique(df[gpi].i)

        for im1 in im1_idx:
            df1 = df[df.i == im1]
            int_ids, i1, i2 = np.intersect1d(df1.g, df2.g, return_indices=True)
            if i1.size > 10:
                coords[(df1.iloc[i1[0]].d, df2.iloc[i2[0]].d)] = Coords(
                    x0 = df1.iloc[i1].x.to_numpy(),
                    x1 = df2.iloc[i2].x.to_numpy(),
                    y0 = df1.iloc[i1].y.to_numpy(),
                    y1 = df2.iloc[i2].y.to_numpy(),
                )
    return coords

def get_one_data_vector(d0r, d1r, r, mfl, r_min, a_max_r, a_max_n, distance_upper_bound1, distance_upper_bound2):
    if r.x0.size < 3:
        return None

    t0r, a0r, p0r = get_triangulation(r.x0, r.y0)
    r0r = np.sqrt(a0r) / p0r
    g0r = (r0r >= r_min) * (a0r <= a_max_r)
    if g0r[g0r].size == 0:
        return None

    rgps_vector = (r.x0, r.y0, d0r, r.x1, r.y1, d1r, t0r, g0r)
    if mfl is None:
        return (None, rgps_vector)

    _, nd0 = mfl.find_nearest(d0r)
    _, nd1 = mfl.find_nearest(d1r)

    if nd0 == nd1:
        #print('nd0 == nd1')
        return None

    xe0r = r.x0[t0r[g0r]].mean(axis=1)
    ye0r = r.y0[t0r[g0r]].mean(axis=1)
    rtree = KDTree(np.vstack([xe0r, ye0r]).T)

    x0n, y0n, ids0 = mfl.get_data(nd0)
    gpi = np.where(np.isfinite(x0n))[0]
    x0n, y0n, ids0 = [i[gpi] for i in [x0n, y0n, ids0]]

    x1n, y1n, ids1 = mfl.get_data(nd1)
    gpi = np.where(np.isfinite(x1n))[0]
    x1n, y1n, ids1 = [i[gpi] for i in [x1n, y1n, ids1]]

    # coordinates of nodes of common elements
    _, ids0i, ids1i = np.intersect1d(ids0, ids1, return_indices=True)
    x0n = x0n[ids0i]
    y0n = y0n[ids0i]
    x1n = x1n[ids1i]
    y1n = y1n[ids1i]

    if x0n.size < 3:
        #print('No common nodes in nextsim')
        return None

    dist, idx = rtree.query(np.vstack([x0n, y0n]).T, distance_upper_bound=distance_upper_bound1)
    idx = np.where(np.isfinite(dist))[0]
    if idx.size < 3:
        #print('No nextsim nodes close to rgps nodes')
        return None

    x0n = x0n[idx]
    y0n = y0n[idx]
    x1n = x1n[idx]
    y1n = y1n[idx]

    t0n, a0n, p0n = get_triangulation(x0n, y0n)
    r0n = np.sqrt(a0n) / p0n
    g0n = (r0n >= r_min) * (a0n <= a_max_n)

    xe0n = x0n[t0n].mean(axis=1)
    ye0n = y0n[t0n].mean(axis=1)

    dist, idx = rtree.query(np.vstack([xe0n, ye0n]).T, distance_upper_bound=distance_upper_bound2)
    idx = np.where(np.isfinite(dist))[0]

    if idx.size == 0:
        #print('No nextsim elements close to rgps nodes')
        return None

    t0n = t0n[idx]
    g0n = g0n[idx]
    _, x0n, y0n = cleanup_triangulation(t0n, x0n, y0n)
    t0n, x1n, y1n = cleanup_triangulation(t0n, x1n, y1n)

    nextsim_vector = (x0n, y0n, pd.Timestamp(nd0), x1n, y1n, pd.Timestamp(nd1), t0n, g0n)

    return nextsim_vector, rgps_vector

In [ ]:
resolution = 10000
# minimal a/p ratio
r_min = 0.15
# maximal area
a_max_r = 200e6
a_max_n = 2. * resolution ** 2
# distance from RGPS nodes to neXtSIM nodes for initial subset
distance_upper_bound1 = 100000
# distance from RGPS elements to neXtSIM elements for final subset
distance_upper_bound2 = resolution * 1.5

idir = './music_matrix/cfg01_m20/sa10free_mat00'

mfl = MeshFileList(idir)

In [ ]:
n = (21 + 31 + 28 + 31) * 2
dst_dates = [dt.datetime(2006,12,10) + dt.timedelta(i/2) for i in range(n)]
print(len(dst_dates), dst_dates[0], dst_dates[-1])
df = pd.read_pickle('./rgps_csv/w07_may_LP.df')


In [ ]:
coords = get_coords_from_traj_new(df, dst_dates[0])
print(len(coords))

In [ ]:
d0rd1r = list(coords.keys())
margin = 1e5
for i, (d0r, d1r) in enumerate([d0rd1r[27]]):
    if 200 < coords[(d0r, d1r)].x0.size < 500:
        print(i, d0r, d1r)
        nextsim_vector, rgps_vector = get_one_data_vector(d0r, d1r, coords[(d0r, d1r)], mfl, r_min, a_max_r, a_max_n, distance_upper_bound1, distance_upper_bound2)
        x, y, i = mfl.get_data(nextsim_vector[2])
        nfile, _ = mfl.find_nearest(nextsim_vector[2])
        m = NextsimMesh(nfile)
        plt.figure(figsize=(10,10))
        plt.triplot(x, y, m.indices, color='k')
        plt.triplot(nextsim_vector[0], nextsim_vector[1], nextsim_vector[6], mask=~nextsim_vector[7], color='b')
        plt.triplot(rgps_vector[0], rgps_vector[1], rgps_vector[6], mask=~rgps_vector[7], color='r')
        plt.xlim([rgps_vector[0].min() - margin, rgps_vector[0].max() + margin])
        plt.ylim([rgps_vector[1].min() - margin, rgps_vector[1].max() + margin])
        plt.gca().set_aspect('equal')
        plt.tight_layout()
        plt.savefig('../tuning_paper_figures/fig03_nextsim_subset.png', dpi=100)
        plt.close()
